In [16]:
import json
import os
import sys
import time
import torch
import numpy as np
import pandas as pd
import esm
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
#from datasets import load_dataset

sys.path.append("../..")
from balm import common_utils
from balm.models.utils import load_trained_model
from balm.configs import Configs
from balm.models import BALM
from balm.metrics import get_ci, get_pearson, get_rmse, get_spearman



In [17]:
CONFIG = {
    'config_filepath': "../../default_configs/balm_peft.yaml",
    'data_path': "../../scripts/notebooks/Data.csv",  # Update with your actual path
    'output_dir': "../../outputs/notebook_runs",
    'checkpoint_dir': "../../outputs/notebook_runs/checkpoints",
    'train_size': 0.2,
    'random_seed': 1234,
    'num_epochs': 50,
    'learning_rate': 0.001,
    'run_name': f"PPB_Affinity_run_{time.strftime('%Y%m%d_%H%M%S')}"
}

# Create output directories
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)

# Setup checkpoint paths
latest_checkpoint_path = os.path.join(CONFIG['checkpoint_dir'], f"{CONFIG['run_name']}_latest.pth")
best_checkpoint_path = os.path.join(CONFIG['checkpoint_dir'], f"{CONFIG['run_name']}_best.pth")

# Set device
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
     DEVICE = 'cpu'
print(f"Using device: {DEVICE}")

# Set random seed
seed = CONFIG['random_seed']
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
np.random.seed(seed)

Using device: cuda


In [18]:
# Define save and load checkpoint functions
def save_checkpoint(model, optimizer, epoch, file_path):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'peft_config': model.model_configs.peft_configs if hasattr(model.model_configs, 'peft_configs') else None

    }
    torch.save(checkpoint, file_path)
    print(f"Checkpoint saved at epoch {epoch} to {file_path}")

def load_checkpoint(model, optimizer,file_path):
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f"Checkpoint loaded from {file_path}, resuming from epoch {start_epoch}")
    return start_epoch

In [19]:
def evaluate_model(model, test_data, pkd_lower_bound, pkd_upper_bound, device, data_min, data_max):
    """Evaluate model on test data."""
    model.eval()
    predictions = []
    labels = []
    protein_seqs = []
    proteina_seqs = []
    
    with torch.no_grad():
        for _, sample in tqdm(test_data.iterrows(), total=len(test_data), desc="Evaluating"):
            # Scale target to cosine similarity range
            cosine_target = 2 * (sample['Y'] - data_min) / (data_max - data_min) - 1
            
            inputs = {
                "protein_sequences": [sample["Target"]],
                "proteina_sequences": [sample["proteina"]],
                "labels": torch.tensor([cosine_target], dtype=torch.float32).to(device)
            }
            
            # Get prediction
            outputs = model(inputs)
            if "cosine_similarity" in outputs:
                cosine_sim = outputs["cosine_similarity"]
                prediction = model.cosine_similarity_to_pkd(
                    cosine_sim,
                    pkd_upper_bound=pkd_upper_bound,
                    pkd_lower_bound=pkd_lower_bound
                )
            else:
                prediction = outputs["logits"]
            
            label = sample["Y"]
            
            predictions.append(prediction.cpu().item())
            labels.append(label)
            protein_seqs.append(sample["Target"])
            proteina_seqs.append(sample["proteina"])
    
    # Calculate metrics
    tensor_labels = torch.tensor(labels)
    tensor_predictions = torch.tensor(predictions)
    
    metrics = {
        'rmse': get_rmse(tensor_labels, tensor_predictions).item(),
        'pearson': get_pearson(tensor_labels, tensor_predictions).item(),
        'spearman': get_spearman(tensor_labels, tensor_predictions).item(),
        'ci': get_ci(tensor_labels, tensor_predictions).item()
    }
    
    return metrics, predictions, labels, protein_seqs, proteina_seqs

def save_evaluation_results(epoch, metrics, predictions, labels, proteins_a, proteins_b, output_dir):
    """Save evaluation results to files."""
    # Create epoch results directory
    epoch_dir = os.path.join(output_dir, f"epoch_{epoch}")
    os.makedirs(epoch_dir, exist_ok=True)
    
    # Save metrics
    metrics_path = os.path.join(epoch_dir, "metrics.json")
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=2)
    
    # Save predictions
    results_df = pd.DataFrame({
        'protein_a': proteins_a,
        'protein_b': proteins_b,
        'true_value': labels,
        'predicted_value': predictions,
        'error': np.array(predictions) - np.array(labels)
    })
    results_path = os.path.join(epoch_dir, "predictions.csv")
    results_df.to_csv(results_path, index=False)
    
    # Create and save regression plot
    plt.figure(figsize=(10, 8))
    ax = sns.regplot(x=labels, y=predictions)
    ax.set_title(f"Epoch {epoch} - Pearson: {metrics['pearson']:.4f}, RMSE: {metrics['rmse']:.4f}")
    ax.set_xlabel("Experimental pKd")
    ax.set_ylabel("Predicted pKd")
    
    # Add identity line
    min_val = min(min(labels), min(predictions))
    max_val = max(max(labels), max(predictions))
    ax.plot([min_val, max_val], [min_val, max_val], 'k--')
    
    plot_path = os.path.join(epoch_dir, "regression_plot.png")
    plt.savefig(plot_path)
    plt.close()
    
    return {
        "metrics_path": metrics_path,
        "results_path": results_path,
        "plot_path": plot_path
    }

# Load Pretrained BALM

In [20]:
# Load config and model for zero-shot
config_filepath = "../../default_configs/balm_peft.yaml"
configs = Configs(**common_utils.load_yaml(config_filepath))

# Custom Dummy Data

Alternative to loading the data from a HF repo, you can also bring in your own custom data.

In [21]:
df = pd.read_csv(CONFIG['data_path'])
print(f"Loaded data with {len(df)} samples")

Loaded data with 11076 samples


In [22]:
# Calculate bounds
data_min = df['Y'].min()
data_max = df['Y'].max()
print(f"Data range: {data_min:.4f} to {data_max:.4f}")

Data range: 1.3188 to 15.6990


# Few shot training

Note: This is a simplified training process, for a more tested approach, please refer to our `balm/trainer.py`

In [23]:
# Split data
train_data, test_data = train_test_split(
    df, 
    train_size=CONFIG['train_size'], 
    random_state=CONFIG['random_seed']
)
print(f"Number of train data: {len(train_data)}")
print(f"Number of test data: {len(test_data)}")

Number of train data: 2215
Number of test data: 8861


In [24]:
# Calculate bounds for normalization
pkd_lower_bound = df['Y'].min()
pkd_upper_bound = df['Y'].max()
print(f"pKd bounds: {pkd_lower_bound:.4f} to {pkd_upper_bound:.4f}")

pKd bounds: 1.3188 to 15.6990


In [25]:
'''
#commenting this because I am normalizing below directly in traing and testing
# Define a function that applies the cosine similarity conversion to a single example
# This is VERY IMPORTANT since BALM uses cosine similarity
def add_cosine_similarity(example, pkd_upper_bound, pkd_lower_bound):
    example['cosine_similarity'] = (
        
        (example['Y'] - pkd_lower_bound) / (pkd_upper_bound - pkd_lower_bound)* 2 - 1
    )
    return example

# Use map to apply the function across the entire dataset
train_data = train_data.apply(lambda x: add_cosine_similarity(x, pkd_upper_bound, pkd_lower_bound), axis=1)
test_data = test_data.apply(lambda x: add_cosine_similarity(x, pkd_upper_bound, pkd_lower_bound), axis=1)
'''

"\n#commenting this because I am normalizing below directly in traing and testing\n# Define a function that applies the cosine similarity conversion to a single example\n# This is VERY IMPORTANT since BALM uses cosine similarity\ndef add_cosine_similarity(example, pkd_upper_bound, pkd_lower_bound):\n    example['cosine_similarity'] = (\n        \n        (example['Y'] - pkd_lower_bound) / (pkd_upper_bound - pkd_lower_bound)* 2 - 1\n    )\n    return example\n\n# Use map to apply the function across the entire dataset\ntrain_data = train_data.apply(lambda x: add_cosine_similarity(x, pkd_upper_bound, pkd_lower_bound), axis=1)\ntest_data = test_data.apply(lambda x: add_cosine_similarity(x, pkd_upper_bound, pkd_lower_bound), axis=1)\n"

In [26]:
# Initialize model for training
configs = Configs(**common_utils.load_yaml(CONFIG['config_filepath']))
# Override learning rate if needed
configs.model_configs.model_hyperparameters.learning_rate = CONFIG['learning_rate']
model = BALM(configs.model_configs)
model = model.to(DEVICE)


In [27]:
import wandb
from torch.optim import AdamW
from tqdm import tqdm
import time


wandb.login()  # Add your key if needed
run = wandb.init(
    project='ppiATTPFT_local_test', 
    entity='hs494-shiv-nadar-university',
    name=CONFIG['run_name'],
    config=CONFIG
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [28]:

# Training loop
NUM_EPOCHS = 20
optimizer = torch.optim.AdamW(
    params=[param for name, param in model.named_parameters() if param.requires_grad],
    lr=configs.model_configs.model_hyperparameters.learning_rate,
)

print(f"Starting training with learning rate: {optimizer.param_groups[0]['lr']}")
print(f"Training on device: {DEVICE}")
print(f"Total training samples: {len(train_data)}")

Starting training with learning rate: 0.001
Training on device: cuda
Total training samples: 2215


In [29]:

#with normalization
# Log hyperparameters
wandb.config.learning_rate = configs.model_configs.model_hyperparameters.learning_rate
wandb.config.num_epochs = NUM_EPOCHS

# Load checkpoint if it exists
start_epoch = 0
# Check both latest and best checkpoints
if os.path.exists(latest_checkpoint_path):
    start_epoch = load_checkpoint(model, optimizer, latest_checkpoint_path)
    print(f"Loaded latest checkpoint, starting from epoch {start_epoch}")
    

In [30]:
# Training loop
NUM_EPOCHS = CONFIG['num_epochs']
start = time.time()
best_loss = float('inf')
best_pearson = -1.0  # Track best performance by Pearson correlation

for epoch in range(start_epoch, start_epoch + NUM_EPOCHS):
    model.train()
    total_loss = 0.0
    
    # Create progress bar for each epoch
    pbar = tqdm(train_data.iterrows(), total=len(train_data), 
                desc=f"Epoch {epoch+1}/{start_epoch + NUM_EPOCHS}", leave=False)
    
    batch_losses = []  # Track individual batch losses
    
    for idx, sample in pbar:
        # Scale to cosine similarity range
        cosine_target = 2 * (sample['Y'] - data_min) / (data_max - data_min) - 1
        
        inputs = {
            "protein_sequences": [sample["Target"]],
            "proteina_sequences": [sample["proteina"]],
            "labels": torch.tensor([cosine_target], dtype=torch.float32).to(DEVICE)
        }

        # Print sequence lengths occasionally
        if idx % 100 == 0:
            print(f"\nSequence lengths - Target: {len(sample['Target'])}, "
                  f"ProteinA: {len(sample['proteina'])}")

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = outputs["loss"]
        loss.backward()
        
        # Gradient norm debugging
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        if idx % 100 == 0:
            print(f"Gradient norm: {grad_norm:.4f}")

        optimizer.step()
        
        current_loss = loss.item()
        batch_losses.append(current_loss)
        total_loss += current_loss
        
        # Update progress bar with current loss
        pbar.set_postfix({
            'batch_loss': f'{current_loss:.4f}',
            'avg_loss': f'{total_loss/(idx+1):.4f}'
        })

    avg_loss = total_loss / len(train_data)
    
    # Print detailed epoch summary
    print(f"\nEpoch [{epoch+1}/{start_epoch + NUM_EPOCHS}] Summary:")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Min Batch Loss: {min(batch_losses):.4f}")
    print(f"Max Batch Loss: {max(batch_losses):.4f}")
    
    # Track best model by training loss
    if avg_loss < best_loss:
        best_loss = avg_loss
        print(f"New best loss achieved: {best_loss:.4f}")

    # Print memory usage if using CUDA
    if torch.cuda.is_available():
        print(f"GPU Memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
        print(f"GPU Memory reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")
    
    # Evaluate on test set after each epoch
    eval_metrics, eval_predictions, eval_labels, protein_a_list, protein_b_list = evaluate_model(
        model, test_data, pkd_lower_bound, pkd_upper_bound, DEVICE, data_min, data_max
    )
    
    # Log metrics to wandb
    wandb.log({
        'epoch': epoch + 1, 
        'train/loss': avg_loss,
        'test/rmse': eval_metrics['rmse'],
        'test/pearson': eval_metrics['pearson'],
        'test/spearman': eval_metrics['spearman'],
        'test/ci': eval_metrics['ci']
    })
    
    # Save evaluation results
    result_paths = save_evaluation_results(
        epoch + 1, 
        eval_metrics, 
        eval_predictions, 
        eval_labels,
        protein_a_list,
        protein_b_list,
        CONFIG['output_dir']
    )
    
    # Log plot to wandb
    wandb.log({"regression_plot": wandb.Image(result_paths["plot_path"])})
    
    # Save latest checkpoint
    save_checkpoint(model, optimizer, epoch+1, latest_checkpoint_path)
    
    # Save best model based on Pearson correlation
    if eval_metrics['pearson'] > best_pearson:
        best_pearson = eval_metrics['pearson']
        save_checkpoint(model, optimizer, epoch+1, best_checkpoint_path)
        print(f"New best model saved with Pearson: {best_pearson:.4f}")

training_time = time.time() - start
print("\nTraining complete!")
print(f"Total training time: {training_time/60:.2f} minutes")
print(f"Average time per epoch: {training_time/(start_epoch + NUM_EPOCHS)/60:.2f} minutes")
print(f"Best loss achieved: {best_loss:.4f}")
print(f"Best Pearson correlation: {best_pearson:.4f}")

Epoch 1/50:   1%|          | 16/2215 [00:02<05:50,  6.27it/s, batch_loss=0.4063, avg_loss=0.0004]


Sequence lengths - Target: 1180, ProteinA: 940


Epoch 1/50:   1%|          | 18/2215 [00:03<08:32,  4.29it/s, batch_loss=0.1621, avg_loss=0.0015]

Gradient norm: 2.0788


Epoch 1/50:  24%|██▍       | 534/2215 [01:21<03:53,  7.21it/s, batch_loss=0.0272, avg_loss=0.0107]

...

Epoch 50/50:  98%|█████████▊| 2161/2215 [05:37<00:07,  7.26it/s, batch_loss=0.0017, avg_loss=0.0040]


Sequence lengths - Target: 266, ProteinA: 165
Gradient norm: 0.0231



Epoch [50/50] Summary:
Average Loss: 0.0172
Min Batch Loss: 0.0000
Max Batch Loss: 0.5419
GPU Memory allocated: 10.70 GB
GPU Memory reserved: 15.54 GB


Evaluating: 100%|██████████| 8861/8861 [21:53<00:00,  6.75it/s]


Checkpoint saved at epoch 50 to ../../outputs/notebook_runs/checkpoints\PPB_Affinity_run_20250407_151953_latest.pth

Training complete!
Total training time: 1283.33 minutes
Average time per epoch: 25.67 minutes
Best loss achieved: 0.0170
Best Pearson correlation: 0.7767


## Test the model!

In [31]:

#with normalization
model = model.eval()

predictions = []
labels = []
start = time.time()

for _, sample in test_data.iterrows():
    # Scale target to cosine similarity range (same as training)
    cosine_target = 2 * (sample['Y'] - data_min) / (data_max - data_min) - 1
    
    # Prepare input (exactly matching training format)
    inputs = {
        "protein_sequences": [sample["Target"]],
        "proteina_sequences": [sample["proteina"]],
        "labels": torch.tensor([cosine_target], dtype=torch.float32).to(DEVICE)
    }
    prediction = model(inputs)["cosine_similarity"]
    prediction = model.cosine_similarity_to_pkd(prediction, pkd_upper_bound=pkd_upper_bound, pkd_lower_bound=pkd_lower_bound)
    label = torch.tensor([sample["Y"]])
    
    # Log prediction and true label to W&B
    wandb.log({"predicted_pKd": prediction.item(), "true_pKd": label.item()})

    print(f"Predicted pKd: {prediction.item()} | True pKd: {label.item()}")
    
    predictions.append(prediction.item())
    labels.append(label.item())
# Log the total time taken
wandb.log({"total_time": time.time() - start})

print(f"Time taken for {len(test_data)} protein-proteina pairs: {time.time() - start}")


Predicted pKd: 10.328910827636719 | True pKd: 10.146302223205566
Predicted pKd: 9.971673965454102 | True pKd: 8.204119682312012
Predicted pKd: 4.571252346038818 | True pKd: 5.0
Predicted pKd: 7.184963226318359 | True pKd: 7.127261161804199
Predicted pKd: 7.522274971008301 | True pKd: 8.602060317993164
Predicted pKd: 9.433027267456055 | True pKd: 9.621602058410645
Predicted pKd: 7.614543914794922 | True pKd: 4.607303142547607
Predicted pKd: 7.821713447570801 | True pKd: 10.0
Predicted pKd: 7.633805274963379 | True pKd: 7.281498432159424
Predicted pKd: 6.3362531661987305 | True pKd: 8.229147911071777
Predicted pKd: 5.448502540588379 | True pKd: 5.040958404541016
Predicted pKd: 6.654026031494141 | True pKd: 5.78515625
Predicted pKd: 13.043770790100098 | True pKd: 10.0
Predicted pKd: 12.064395904541016 | True pKd: 10.268411636352539
Predicted pKd: 4.144892692565918 | True pKd: 4.958607196807861
Predicted pKd: 10.35123348236084 | True pKd: 9.772112846374512
Predicted pKd: 6.239045143127441 

In [32]:
# Final evaluation using the best model
print("\n--- Final Evaluation using Best Model ---")

# Load the best model
load_checkpoint(model, optimizer, best_checkpoint_path)
model.eval()

# Perform full evaluation
final_metrics, final_predictions, final_labels, final_protein_a, final_protein_b = evaluate_model(
    model, test_data, pkd_lower_bound, pkd_upper_bound, DEVICE, data_min, data_max
)

# Print final metrics
print("\nFinal Evaluation Metrics:")
print(f"RMSE: {final_metrics['rmse']:.4f}")
print(f"Pearson: {final_metrics['pearson']:.4f}")
print(f"Spearman: {final_metrics['spearman']:.4f}")
print(f"CI: {final_metrics['ci']:.4f}")

# Log final metrics to W&B
wandb.log({
    "final/rmse": final_metrics['rmse'],
    "final/pearson": final_metrics['pearson'],
    "final/spearman": final_metrics['spearman'],
    "final/ci": final_metrics['ci']
})

# Save final results
final_paths = save_evaluation_results(
    "final", 
    final_metrics, 
    final_predictions, 
    final_labels,
    final_protein_a,
    final_protein_b,
    CONFIG['output_dir']
)

# Log final plot to wandb
wandb.log({"final_regression_plot": wandb.Image(final_paths["plot_path"])})

# Create additional visualization
plt.figure(figsize=(12, 8))
plt.scatter(final_labels, final_predictions, alpha=0.6)
plt.plot([min(final_labels), max(final_labels)], [min(final_labels), max(final_labels)], 'r--')
plt.xlabel("Experimental pKd")
plt.ylabel("Predicted pKd")
plt.title(f"Final Results - Pearson: {final_metrics['pearson']:.4f}, RMSE: {final_metrics['rmse']:.4f}")
plt.tight_layout()

# Save and log
final_scatter_path = os.path.join(CONFIG['output_dir'], "final_scatter_plot.png")
plt.savefig(final_scatter_path)
wandb.log({"final_scatter_plot": wandb.Image(final_scatter_path)})
plt.close()

# End wandb run
wandb.finish()

print(f"\nAll results saved to: {CONFIG['output_dir']}")
print("Training and evaluation complete!")


--- Final Evaluation using Best Model ---


C:\Users\hs494\AppData\Local\Temp\ipykernel_39204\126795426.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(file_path)


Checkpoint loaded from ../../outputs/notebook_runs/checkpoints\PPB_Affinity_run_20250407_151953_best.pth, resuming from epoch 34


Evaluating: 100%|██████████| 8861/8861 [22:04<00:00,  6.69it/s]



Final Evaluation Metrics:
RMSE: 1.3892
Pearson: 0.7767
Spearman: 0.7730
CI: 0.7929


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
final/ci,▁
final/pearson,▁
final/rmse,▁
final/spearman,▁
predicted_pKd,▃▇▄▂▅▄▅▁▅▄▂▆█▂▂▅▂▄▂▄▂▅▃▁▄▁▁▃▃▄▃▅▄▄▄▂▅▂▁▂
test/ci,▁▃▄▅▅▆▆▇▇▇▇▇▇▇█▇▇▇▇▇███▇████▇██▇▇█▇██▇██
test/pearson,▁▄▄▅▆▆▆▇▇▇▇▇▇▇█▇▇▇▇▇▇███████▇██▇██▇██▇██
test/rmse,█▄▄▃▃▃▃▂▂▂▂▂▃▂▁▂▂▂▂▂▁▁▁▂▁▁▁▁▂▁▁▂▂▁▁▁▁▂▁▁
test/spearman,▁▄▄▅▆▆▇▇▇▇▇▇▇▇█▇▇█▇█▇███▇████▇▇█▇▇▇██▇██
total_time,▁



All results saved to: ../../outputs/notebook_runs
Training and evaluation complete!
